In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
anime_pivot = pd.read_csv('csv/2020/anime_pibot_2020.csv')

In [ ]:
from scipy.sparse import csr_matrix
#Creating a sparse matrix
anime_user_matrix = csr_matrix(anime_pivot.values)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Calculate the cosine similarity between users
anime_similarity = cosine_similarity(anime_user_matrix)

In [ ]:
# Generate recommendations using the similarity scores
norm = np.abs(anime_similarity).sum(axis=1)
predicted_anime_ratings = anime_similarity.dot(anime_pivot.values) / norm[:, np.newaxis]

In [ ]:
# Write a function to find top k similar anime from anime_pivot
def recommend_item_base(anime_name, k=10):
    anime_index = anime_pivot.index.get_loc(anime_name)
    anime_indices = np.argsort(anime_similarity[anime_index])[-k:][::-1]
    similar_anime_ids = anime_pivot.index[anime_indices]
    return similar_anime_ids

In [ ]:
recommend_item_base('Naruto')